<a href="https://colab.research.google.com/github/tatooland/keras/blob/master/KERAS_CNN_INTEL_IMAGE_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##define GOOGLE DRIVER downloader

In [1]:
!pip install -U -q PyDrive
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def downloadFromGDriver(fileId):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  fileName = fileId + '.zip'
  downloaded = drive.CreateFile({'id': fileId})
  downloaded.GetContentFile(fileName)
  ds = ZipFile(fileName)
  ds.extractall()
  os.remove(fileName)
  print('Extracted zip file ' + fileName)


     |████████████████████████████████| 993kB 3.5MB/s 


## define uncompress zip file processing

In [0]:
import os
from zipfile import ZipFile
def uncompressZipFile(fileName):
   ds = ZipFile(fileName)
   ds.extractall()
   os.remove(fileName)
   print("process complete")

##define image iterator

In [0]:
import os
def iteratorImgFolder(folder, callback):
  g = os.walk(folder)
  ary = []
  for path, _, fileList in g:
    for fileName in fileList:
      callback(os.path.join(path, fileName), ary)
  return ary

## define image reader

In [4]:
from keras.preprocessing.image import load_img, img_to_array
def readImgAsArray(fileName, ary):
  img = load_img(fileName)
  format_img = img_to_array(img)
  print(format_img.shape)
  ary.extend(format_img)

Using TensorFlow backend.


In [5]:
#download image data from google driver
fileId = "1Uy-VRPQlrjfPHkJlZEgxUH-_4vMI1CUn"
downloadFromGDriver(fileId)
uncompressZipFile("seg_pred.zip")
uncompressZipFile("seg_test.zip")
uncompressZipFile("seg_train.zip")

W0626 01:05:34.298465 139833822234496 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0626 01:05:35.453958 139833822234496 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py

Extracted zip file 1Uy-VRPQlrjfPHkJlZEgxUH-_4vMI1CUn.zip
process complete
process complete
process complete


In [0]:
%pwd

'/content'

##define image pre processing

In [0]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

def readImgDataAsFlow(train_dir, test_dir, validation_dir, target_size=(150, 150)):
  trainGenerator = ImageDataGenerator(data_format="channels_last")
  testGenerator = ImageDataGenerator(data_format="channels_last")
  validationGenerator = ImageDataGenerator(data_format="channels_last")
  trainFlow = trainGenerator.flow_from_directory(train_dir, class_mode='categorical', target_size=target_size)
  testFlow = testGenerator.flow_from_directory(test_dir, class_mode='categorical', target_size=target_size)
  validationFlow = validationGenerator.flow_from_directory(validation_dir, class_mode='categorical', target_size=target_size)
  return trainFlow, testFlow, validationFlow
  

## define model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD

def defineModel():
  model = Sequential()
  
  #(150-3)/1 + 1 = 148 (148, 148, 32)
  model.add(Conv2D(32, (3, 3), padding='valid', activation='relu', input_shape=(150, 150, 3)))
  #(74, 74, 32)
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  #(74-3)/1 + 1 = 72 (72, 72, 64)
  model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
  #(36, 36, 64)
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  #(36-3)/1 + 1 = 34 (34, 34, 128)
  model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
  #(17, 17, 128)
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))
  
  model.add(Flatten())
#  model.add(Dense(128, activation='relu'))
#  model.add(Dropout(0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(6, activation='softmax'))
  return model


def defineVgg16():
  #output: 75, 75, 64
  model = Sequential()
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(150, 150, 3)))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Flatten())
  model.add(Dense(4096, activation='relu'))
  model.add(Dense(4096, activation='relu'))
  model.add(Dense(6, activation='softmax'))
  
  return model
  
  
  
  

## run main fun

In [12]:
import matplotlib.pyplot as plt

import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16



trainFlow, testFlow, validationFlow = readImgDataAsFlow("/content/seg_train", "/content/seg_test", "/content/seg_pred")


print(trainFlow)
print(testFlow)
print(validationFlow)

#ary = iteratorImgFolder("/content/seg_train/buildings/", readImgAsArray)
#print(np.shape(ary))
#img = load_img("/content/seg_train/buildings/10029.jpg")
#format_img = img_to_array(img)
#print(format_img.shape)

model = defineVgg16()
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit_generator(
  trainFlow,
  steps_per_epoch=2000,
  epochs = 1000,
  validation_data=testFlow,
  validation_steps=800
)



Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Found 0 images belonging to 0 classes.
Epoch 1/1000


ValueError: ignored

In [0]:
%ls seg_pred

10004.jpg  13296.jpg  16412.jpg  19543.jpg  22613.jpg  3947.jpg  6943.jpg
10005.jpg  13297.jpg  16415.jpg  19549.jpg  22615.jpg  3953.jpg  6948.jpg
10012.jpg  13299.jpg  16424.jpg  19557.jpg  22618.jpg  3962.jpg  6950.jpg
10013.jpg  132.jpg    16428.jpg  19558.jpg  22621.jpg  3966.jpg  6951.jpg
10017.jpg  13302.jpg  16429.jpg  19560.jpg  22622.jpg  3970.jpg  6952.jpg
10021.jpg  13303.jpg  16436.jpg  19561.jpg  22627.jpg  3975.jpg  6957.jpg
10034.jpg  13305.jpg  16442.jpg  19562.jpg  22630.jpg  3976.jpg  695.jpg
10038.jpg  1330.jpg   16444.jpg  19564.jpg  22638.jpg  3979.jpg  6964.jpg
1003.jpg   13316.jpg  16445.jpg  19568.jpg  22641.jpg  3981.jpg  6965.jpg
10040.jpg  13318.jpg  16446.jpg  19569.jpg  22649.jpg  3982.jpg  6966.jpg
10043.jpg  13321.jpg  16447.jpg  19570.jpg  22651.jpg  3983.jpg  6974.jpg
10045.jpg  13325.jpg  16449.jpg  19572.jpg  22658.jpg  3986.jpg  6980.jpg
10047.jpg  13326.jpg  1644.jpg   19575.jpg  22659.jpg  3987.jpg  6988.jpg
10048.jpg  13329.jpg  16454.jpg  19577.